In [1]:
import nltk
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize 
#nltk.download('wordnet')
#nltk.download('stopwords')
from nltk.corpus import wordnet #wornet loaded
import re
import json
from flask import Flask,render_template, request, redirect, url_for
from flask import jsonify, current_app
from flask_cors import CORS

#Methode definition
# https://stackoverflow.com/questions/1883980/find-the-nth-occurrence-of-substring-in-a-string
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    print("le debut",start,n)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    print(start)
    return start

def get_synsets(w):
    syns = wordnet.synsets(w)
    return syns

def get_definitions(syns):
    return syns.definition()

def get_label(syns):
    lemmas = syns.lemmas()
    name = lemmas[0].name()
    return name

def get_class(syns):
    return syns.pos()

def get_resource(syns):
    return syns.offset()  #id for a synset in wordnet db

def get_source(syns):
    return syns.name()

#TODO what if the text is used multiple times?
def get_offset(w,text,nth):
    offset = find_nth(text, w, nth)
    return offset

def stop_words_filtering(text):
    reg_exp = r"[a-zA-Z0-9_-]+"  #Reguläre Ausdruck, der alle erlaubten Sylabeln enthält. Von "a" bis "z"
                            #von "A" bis "Z"   
    #reg_exp2 = r"[a-zA-Z0-9]+"
    stop_words  = set(stopwords.words('english')) #stopwörter für englisches Text
    word_tokens = []
    #word_token2 = []
    a = re.compile(reg_exp)
    #b = re.compile(reg_exp2)
    word_tokens = word_tokens + a.findall(text) #löscht die Satzzeichnen im Text
    filtered_sentence = filtered_sentence2 =  [w for w in word_tokens if not w in stop_words] #löscht die Stopp-Wörter im Text
    #word_tokens2 = word_tokens2 + b.findall(text) #für die wörter wie pet_food pet-food (-,_ löschen)
     
    return filtered_sentence 
#filtered_sentence = [] 
    #for w in word_tokens: 
      #  if w not in stop_words: 
    #        filtered_sentence.append(w)

#Nested dictionary
def build_ressources_candidates(word,syns,offset):
    resourcedict = {}
    resourcedict["description"] = get_definitions(syns)
    resourcedict["label"]       = get_label(syns)
    resourcedict["offset"]      = offset
    resourcedict["resource"]    = get_resource(syns)
    resourcedict["source"]      = get_source(syns)
    resourcedict["text"]        = word
    resourcedict["pos"]         = get_class(syns) #pos(part of speech oder word class)
    return resourcedict   
    
def build_resources(word,offset):
    resources = []
    synsets = get_synsets(word)
    if "-" in word:
        tokens = word.split("-")
        for w in tokens:
            synsets.extend(get_synsets(w))
    elif "_" in word:
        tokens = word.split("_")
        for w in tokens:
            synsets.extend(get_synsets(w))    
    for syns in synsets:
        resources.append(build_ressources_candidates(word,syns,offset))
    return resources

def build_annot_candidates(word,text,current_word_occurence_count):
        offset = get_offset(word,text,current_word_occurence_count)
        candidate = {}
        candidate["offset"]                = offset
        candidate["resource_candidates"]  = build_resources(word,offset)
        candidate["text"]                  = word
        return candidate
def build_annot(text):
   
    annotDict = {}
    annot = []
    annotDict["annotation_candidates"]   = annot
    annotDict["text"]                    = text
    Woerter = stop_words_filtering(text)
    wordCount = {}
    print('currentWords:',Woerter)
    #print(Woerter)
    for w in Woerter:
        find = 0
        if not bool(wordCount): #wenn dictionary leer ist
            wordCount[w] = 0
            wordCount[w] = wordCount[w] + 1
            currentWordCount = wordCount[w]
            print("Start:",w)
        else:
            #if not (w in wordCount.keys()):
            #wordCount[w] = 0
            for key in wordCount.copy():
                #if (key.find(w) == -1):   
                if (key.find(w) != -1):
                    print("ich bin da find ist true:", key,w)
                    wordCount[key] = wordCount[key] + 1
                    currentWordCount = wordCount[key]
                    find = 1
                   # print("after",wordCount[w])
                    break
            #print("ich bin da:", key,w)
            if find != 1:
                print("not find: ", w)
                wordCount[w] = 0
                wordCount[w] = wordCount[w] + 1
                currentWordCount = wordCount[w]
                   # print("actual",currentWordCount)
                #print("nach",wordCount[w])
        annot.append(build_annot_candidates(w,text, currentWordCount))
                    
        #if not (w in wordCount.keys()):
            #wordCount[w] = 0
        
            #annot.append(build_annot_candidates(w,text, currentWordCount))
    print(wordCount)
    return annotDict

    
    
    

In [2]:
#json.dumps(build_annot("say that you say"))
#import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
#d = stop_words_filtering("I am a winners, but not a looser 9!")
#print(d)
#import sys
#sys.stdout = open('synset.png', 'w')
#print(get_synsets('Framework'))

  # print()
#annotation = build_annot("pet-food,petfood and pet")
#dic = {"pet":"food"}
#text = "pet-food"
#separator = ''
#print(bool(dic))
#print(text.startswith("pet"))
#print(text.find("food"))
#print(text.index("food"))
#print(enumerate(text))
#for index, word in enumerate(nltk.word_tokenize(text)):
#    print(index, word)
#print(annotation)


In [ ]:
app = Flask(__name__)
CORS(app)
@app.route('/')
def index():
  return 'Server Works done!'
    
@app.route('/annotApi/<string:text>')
def annotsFunction2(text):
    annotation = build_annot(text)
    #print(json.dumps(annotation))
    return json.dumps(annotation)

    
if __name__ == '__main__':
    with app.app_context():
   # app.debug = True
        from werkzeug.serving import run_simple
        run_simple('localhost', 4000, app)

 * Running on http://localhost:4000/ (Press CTRL+C to quit)


currentWords: ['find', 'lost', 'livestock']
Start: find
le debut 3 1
3
not find:  lost
le debut 8 1
8


127.0.0.1 - - [27/Jan/2020 16:29:22] "GET /annotApi/to%20find%20lost%20livestock HTTP/1.1" 200 -


not find:  livestock
le debut 13 1
13
{'find': 1, 'lost': 1, 'livestock': 1}
currentWords: ['device', 'could', 'used', 'detective', 'manner', 'say', 'could', 'useful', 'predicting', 'behavioral', 'patterns', 'say', 'criminal', 'offenders', 'students', 'perform', 'research', 'perhaps', 'tracking', 'physical', 'lives', 'athletes', 'top', 'achieving', 'businessman', 'companies', 'isolate', 'expected', 'behaviors', 'employees', 'anticipate', 'employees', 'move', 'work', 'day', 'establish', 'fair', 'true', 'standards']
Start: device
le debut 4 1
4
not find:  could
le debut 11 1
11
not find:  used
le debut 20 1
20
not find:  detective
le debut 30 1
30
not find:  manner
le debut 40 1
40
not find:  say
le debut 58 1
58
ich bin da find ist true: could could
le debut 11 2
70
not find:  useful
le debut 79 1
79
not find:  predicting
le debut 89 1
89
not find:  behavioral
le debut 100 1
100
not find:  patterns
le debut 111 1
111
ich bin da find ist true: say say
le debut 58 2
123
not find:  crimina

127.0.0.1 - - [27/Jan/2020 16:29:57] "GET /annotApi/the%20device%20could%20be%20used%20in%20a%20detective%20manner%20that%20is%20to%20say%20that%20it%20could%20be%20useful%20to%20predicting%20behavioral%20patterns%20of%20say%20criminal%20offenders,%20students,%20to%20perform%20research%20perhaps%20by%20tracking%20the%20physical%20lives%20of%20athletes%20or%20top%20achieving%20businessman%20or%20for%20companies%20to%20isolate%20expected%20behaviors%20of%20their%20employees%20and%20how%20they%20anticipate%20employees%20to%20move%20during%20a%20work%20day%20to%20establish%20fair%20and%20true%20standards HTTP/1.1" 200 -


{'device': 1, 'could': 2, 'used': 1, 'detective': 1, 'manner': 1, 'say': 2, 'useful': 1, 'predicting': 1, 'behavioral': 1, 'patterns': 1, 'criminal': 1, 'offenders': 1, 'students': 1, 'perform': 1, 'research': 1, 'perhaps': 1, 'tracking': 1, 'physical': 1, 'lives': 1, 'athletes': 1, 'top': 1, 'achieving': 1, 'businessman': 1, 'companies': 1, 'isolate': 1, 'expected': 1, 'behaviors': 1, 'employees': 2, 'anticipate': 1, 'move': 1, 'work': 1, 'day': 1, 'establish': 1, 'fair': 1, 'true': 1, 'standards': 1}
